# Security Basics

This is an attempt to learn basic concepts of security by application using Python librar(ies). We'll review the typical Problems addressed by security and demonstrate implemetations to address them, and introducing security terms alongside this discussion.

## 1. Confidentiality

Two friends, Alice and Bob, want to exchange important messages, and they don't want anyone else to 'listen' to their conversation (need for secrecy). They can use **Encryption** to convert *plaintext (pt)* of their messages into *ciphertext (ct)*

## 1.1 Symmetric Encryption

If the same key is used for both Encryption and Decryption, then the process is called symmetric.

In [4]:
from cryptography.fernet import Fernet

plaintext = b"This is my secret message"    # Store as type 'bytes'

# TODO Use AES algorithm for symmetric encryption

# Use Fernet to perform encryption - the recommended way for Python: https://github.com/fernet/spec/blob/master/Spec.md
# Named after famous and important Fernet's Little Theorem: https://www.britannica.com/science/Fermats-theorem
sym_priKey = Fernet.generate_key()              # Key generation
f = Fernet(sym_priKey)                          # Fernet object that provides crypto services
ciphertext = f.encrypt(plaintext)           # Encryption

decryptedText = f.decrypt(ciphertext)       # Decryption

assert decryptedText == plaintext           # Assertion to validate decryption returns the original message

Symmetric key (or private key) encryption is simpler and useful to encrypt large messages, however, the challenge is keep the 'key' secret. This is where the assymetric key (or public key) encryption is useful.

## 1.2 Assymetric Key encryption

A public key is used to encrypt the message, and corresponding private key to decrypt. Mathematically, it is required that for a given public key (known to everyone), there's exactly a single private key that can perform the decryption, and for all other keys, the decryption will fail.

This allows anyone to secretly send a message that can only be decrypted by the recipient.

In [5]:
# Source: https://medium.com/@ashiqgiga07/asymmetric-cryptography-with-python-5eed86772731

#Importing necessary modules
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA

#Generating private key (RsaKey object) of key length of 1024 bits
rsa_strength = 1024
asym_priKey = RSA.generate(rsa_strength)

#Generating the public key (RsaKey object) from the private key
asym_pubKey = asym_priKey.publickey()

#Instantiating PKCS1_OAEP object with the public key for encryption
cipher = PKCS1_OAEP.new(key=asym_pubKey)

#Encrypting the message with the PKCS1_OAEP object
ciphertext = cipher.encrypt(plaintext)

#Instantiating PKCS1_OAEP object with the private key for decryption
decrypt = PKCS1_OAEP.new(key=asym_priKey)

#Decrypting the message with the PKCS1_OAEP object
decryptedtext = decrypt.decrypt(ciphertext)

assert decryptedText == plaintext           # Assertion to validate decryption returns the original message

<class 'Crypto.PublicKey.RSA.RsaKey'> <class 'Crypto.PublicKey.RSA.RsaKey'>
b'yQQt\xa9.pC<\x1e\x0e\xa5Q.$\xbf(\xf9Yd\xb5^\xffB\x98\x00\x8eEB\x05\xfd\xb4\x0b\x9a\xb0\xbb\xc0\xf1\xbf|\xde\x0e\xf3^.\xf6L\xa1\xfa\xed\xcc\x83\x13-\xfaX\xa4\xa4!R\x99?v\xa5\x91h\xc0\xda\r\x08\xdbZ\xc1\xa4Y[Z\x8b\xd5k\xb8\x05\xefM\xd7.a\xe1\xa2~\x04\x15\x05\xfe\xec0W\x06NAu\\S\xcb\xb5B\xcdh\x8dr\xf7\xdf\xe0\x1f\xb5\xa3\xee[\xad\x01-\xe9~\x02=<\xb1\xfe'
b'This is my secret message'


{ct, ct_enc_keyAs seen above, the process for confidential messaging (using public key cryptography) is as follows:

1. Obtain a private key
2. Obtain a public key corresponding to private key
3. Each recipient shares his public key with everyone (more on this later in authentication)
4. Sender encrypts the message using recipient's public key, so that only the intended recipient can decrypt it.
5. Recipient uses private key to decrypt and obtain the message

However, the process of encryption - decryption ONLY provides secrecy, but not the authentication, i.e., a malicious 'middle person' M can change the message sent to the recipient (B), even if M can't decrypt what the sender A was sending to B.

## 2. Authentication

Public key cryptography can be used to 'sign' the message to establish an identity of the sender, using the following process (that provides both confidentiality and authentication):

In [ ]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

# Recipient generates key-pair and publishes public key
class Recipient:
    def __init__(self, rsa_strength):
        self.recPriKey = RSA.generate(rsa_strength)

    def GetPublicKey():
        return self.recPriKey.publickey()

    def ReceiveEncryptedSignedMessage(ct, ct_encKey, signed_digest, signPubKey):
        # Decrypted signed digest
        decrypt = PKCS1_OAEP.new(key=signPubKey)
        decr_digest = decrypt.decrypt(signed_digest)

        # Form digest of {ct, ct_encKey}
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(ct)
        digest.update(ct_encKey)
        digest.finalize()

        # Compare digests
        if(decr_digest == digest):
            # Message authenticated, now decrypt

            # Decrypt the symmetric encryption key used to encrypt the message
            decrypt = PKCS1_OAEP.new(key=self.recPriKey)
            decr_encKey = decrypt.decrypt(ct_encKey)

            # Decrypt the message and return
            f = Fernet(decr_encKey)
            pt = f.decrypt(ct)

            return pt

class Sender:
    def __init__(self, rsa_strength):
        self.signPriKey = RSA.generate(rsa_strength)
    
    def SendEncryptedSignedMessage(plaintext, recPubKey):
        # Encrypt plaintext using a private key algorithm to generate ciphertext
        encKey = Fernet.generate_key()
        f = Fernet(encKey)
        ct = f.encrypt(plaintext)

        # TODO plaintext and encKey should be quickly hidden to avoid memory based attacks

        # Encrypt the encKey using recipient's public key and a public key encryption (e.g. RSA)
        cipher_enc = PKCS1_OAEP.new(key=recPubKey)
        ct_encKey = cipher_enc.encrypt(encKey)

        # Generate hash of the ciphertext & encypted encryption key (ct + ct_encKey)
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(ct)
        digest.update(ct_encKey)
        digest.finalize()
        print(digest)

        # Use signing private key to sign the digest and generate signature
        cipher_sign = PKCS1_OAEP.new(key=self.signPriKey)
        signed_digest = cipher_sign.encrypt(digest)
        signPubKey = self.signPriKey.public_key()

        # Send the encrypted and signed message
        return [ct, ct_encKey, signed_digest, signPubKey]

# Create Recipient and Sender objects
recipient = Recipient(rsa_strength)
sender = Sender(rsa_strength)

# Get Recipient's public key
recipientPublicKey = recipient.GetPublicKey()

txMessage = sender.SendEncryptedSignedMessage(plaintext, recipientPublicKey)

decryptedText = recipient.ReceiveEncryptedSignedMessage(txMessage)

assert decryptedText == plaintext           # Assertion to validate decryption returns the original message